In [112]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.ensemble import RandomForestClassifier

In [5]:
X=pd.read_csv('x_train.csv',delimiter=";",header=None,names=['f'+str(i) for i in range (223)])
y=pd.read_csv('y_train.csv',names=['class'])
y=y['class']
X_test=pd.read_csv('x_test.csv',delimiter=";",header=None,names=['f'+str(i) for i in range (223)])

In [9]:
param_grid = {
    'learning_rate': [0.01, 0.05,0.02, 0.03,0.04],
    'n_estimators': [200,300,400,500,600],
    'num_leaves': [8],
    #'max_depth': [3,4],
    'boosting_type' : ['gbdt'],
    'objective' : ['binary'],
    'seed' : [777],
    'colsample_bytree' : [0.7,0.8,0.9,1],
    'subsample' : [0.7,0.8,0.9,1],
    'reg_alpha' : [0,0.1,0.5,1],
    'reg_lambda' : [0,1,2,5,6,7,9,10],
    'is_unbalance' : [True,False],
}

In [30]:
def modelfit(clf, printFeatureImportance=True, cv_folds=5):
    cv_score = cross_val_score(clf, X, y, cv=cv_folds)
    print ("CV Score : Mean - %.7g | Std - %.7g | Min - %.7g | Max - %.7g" % (np.mean(cv_score),np.std(cv_score),np.min(cv_score),np.max(cv_score)))
      #Print Feature Importance:
    if printFeatureImportance:
        feat_imp = pd.Series(clf.feature_importances_, X.columns).sort_values(ascending=False)
        feat_imp.plot(kind='bar', title='Feature Importances')
        plt.ylabel('Feature Importance Score')


In [ ]:
Подбираем n_estimators

In [57]:
estimator = lgb.LGBMClassifier(learning_rate=0.08)
param_grid = {
    #'learning_rate': [0.01, 0.05],
    'n_estimators':  np.arange(10,101,10)
}

ngrid=2

gbm = GridSearchCV(estimator, param_grid=param_grid,cv=5,verbose=True)
gbm.fit(X, y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed: 11.4min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=LGBMClassifier(boosting_type='gbdt', colsample_bytree=1, drop_rate=0.1,
        is_unbalance=False, learning_rate=0.08, max_bin=255, max_depth=-1,
        max_drop=50, min_child_samples=10, min_child_weight=5,
        min_split_gain=0, n_estimators=10, nthread=-1, num_leaves=31,
        ob...     subsample_for_bin=50000, subsample_freq=1, uniform_drop=False,
        xgboost_dart_mode=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': array([ 10,  20,  30,  40,  50,  60,  70,  80,  90, 100])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=True)

In [58]:
gbm.best_params_

{'n_estimators': 60}

Подбираем max_depth и min_child_weight

In [71]:
estimator = lgb.LGBMClassifier(learning_rate=0.08,n_estimators=60)
param_test = {
 'max_depth':[9,11,13],
 'min_child_weight':[5,9,11]
}
#'min_data_in_leaf':np.arange(20,1001,50)
gbm = GridSearchCV(estimator,param_grid=param_test,cv=5,verbose=True)
gbm.fit(X, y)
gbm.best_params_

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:  5.8min finished


{'max_depth': 9, 'min_child_weight': 5}

Подбираем subsample and colsample_bytree

In [78]:
estimator = lgb.LGBMClassifier(learning_rate=0.08,n_estimators=60,max_depth=9,min_child_weight=5,subsample=0.8)
param_test = {
 'colsample_bytree':[i/100.0 for i in range(10,40,10)]
}
gbm = GridSearchCV(estimator,param_grid=param_test,cv=5,verbose=True)
gbm.fit(X, y)
gbm.best_params_
gbm.best_estimator_

Fitting 5 folds for each of 3 candidates, totalling 15 fits


KeyboardInterrupt: 

In [79]:
estimator = lgb.LGBMClassifier(learning_rate=0.08,n_estimators=60,max_depth=9,min_child_weight=5,subsample=0.8,colsample_bytree=0.2)
param_test = {
 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
gbm = GridSearchCV(estimator,param_grid=param_test,cv=5,verbose=True)
gbm.fit(X, y)
gbm.best_params_

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:   34.3s finished


{'reg_alpha': 1e-05}

In [80]:
estimator = lgb.LGBMClassifier(learning_rate=0.01,max_depth=9,min_child_weight=5,subsample=0.8,colsample_bytree=0.2,reg_alpha=1e-05)
param_test = {
 'n_estimators':[1000, 1500,2000,3000,5000]
}
gbm = GridSearchCV(estimator,param_grid=param_test,cv=5,verbose=True)
gbm.fit(X, y)
gbm.best_params_

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed: 20.5min finished


{'n_estimators': 1500}

In [82]:
est=gbm.best_estimator_

In [88]:
fi=est.feature_importances_
fi_pf=pd.DataFrame(data=fi,columns=['fi_rank'],index=X.columns)

In [101]:
fi_pf=fi_pf.sort_values(['fi_rank'],ascending=False)

In [107]:
fi_cols=list(fi_pf.index)

In [114]:
X

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f213,f214,f215,f216,f217,f218,f219,f220,f221,f222
0,-1.351735,1.502242,2.049512,0.437597,0.243818,4.982503,-1.493039,1.868572,1.435876,3.594717,...,3.706007,-0.789066,4.193481,2.803058,0.685003,3.204492,3.528085,4.877951,0.937529,0.730470
1,2.810473,1.312591,1.392652,0.163840,0.654384,-2.725782,3.195347,1.910067,0.321657,0.256354,...,-1.442361,4.913119,-1.182302,1.540444,0.563015,1.976227,0.854290,0.490507,-0.190323,1.244596
2,2.328788,-1.928459,-2.064532,0.731323,0.117712,-1.567757,2.961654,-3.013697,0.185553,5.821898,...,-1.373806,1.991433,-2.514600,-2.660716,1.124836,-2.993835,-1.519144,1.851507,0.058385,0.706138
3,-0.128106,-2.072688,-2.407602,0.978550,0.111442,2.608775,-1.299084,-1.919077,1.566608,2.349346,...,0.295274,-2.896897,-1.639309,-2.186183,-0.097270,-3.085381,-0.802771,4.713787,0.943972,-0.320782
4,1.886821,0.757923,-0.097547,0.465719,0.621116,-0.332968,2.446465,-0.143173,0.032383,0.837508,...,-0.340020,1.320251,0.447317,-0.103243,1.285391,-0.445947,-0.459055,-2.744683,0.620512,1.166808
5,-1.093613,1.747583,2.499609,0.366799,0.888956,3.371590,-1.416785,2.515114,0.654840,0.838568,...,3.065135,-1.000136,4.132015,2.437985,-0.076527,3.986756,1.832630,-0.537049,0.648463,0.570630
6,0.717601,-1.307117,-2.156820,0.337006,0.071711,1.920241,0.730820,-2.064205,0.559461,1.882081,...,1.331225,-0.729551,-0.379827,-1.800957,1.653230,-1.305788,-1.261134,2.268844,1.676191,0.938412
7,4.600603,-1.605449,-1.889961,0.945001,0.637761,-0.853413,5.844666,-2.233650,0.518793,4.436753,...,-2.469902,6.960248,-2.624123,-1.596716,2.459454,-2.181559,0.414740,8.020731,-0.697364,1.442853
8,-0.842231,2.782335,3.072997,0.314701,0.344247,1.278883,-1.316395,3.502783,0.748349,1.575257,...,2.877581,-0.484909,3.794395,2.806133,-0.265576,4.408394,2.000839,-1.425723,0.322877,-0.309355
9,-0.712364,0.531405,0.466771,0.123207,0.588291,1.935748,-1.307892,-0.374183,0.746420,5.737873,...,1.934958,-1.314380,1.059568,-0.058500,0.763539,1.085616,0.395652,2.683181,1.273965,-0.172293


In [116]:
ftr_set=['f11', 'f96', 'f138', 'f156', 'f76', 'f131', 'f185', 'f45'] 
clf = lgb.LGBMClassifier(learning_rate=0.01,max_depth=9,min_child_weight=5,subsample=0.8,colsample_bytree=0.2,reg_alpha=1e-05)
cv_score = cross_val_score(clf, X[ftr_set], y, cv=5).mean()
print (cv_score)

0.492375853059
